In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc
import sys

import os 
import sys
import pandas as pd
import scanpy as sc
from scipy.sparse import csr_matrix

from nicheformer.data.constants import DefaultPaths, ObsConstants, UnsConstants, VarConstants, AssayOntologyTermId, SexOntologyTermId, OrganismOntologyTermId, TissueOntologyTermId, SuspensionTypeId
from nicheformer.data.validate import validate

In [4]:
import sys
sys.path.append("/home/ec2-user/SageMaker/functions")
from preprocessing import *


In [6]:
#if len(sys.argv)==1:
path = "./our_data_LC_NE"
#else:
#    path = sys.argv[1]

# raw_path = f"{path}"
# preprocessed_path = f"{path}/preprocessed"

# if not os.path.exists(f"{preprocessed_path}"):
#     os.mkdir(f"{preprocessed_path}")

adata = sc.read_h5ad(f"{path}/adata_mer.h5ad")


# setting MERFISH for now until an official ontology term is released
assay = str(AssayOntologyTermId.MERFISH_SPATIAL.value)
sex = str(SexOntologyTermId.UNKNOWN.value)
organism = str(OrganismOntologyTermId.MOUSE.value)
organism_validator = "mouse"
tissue = str(TissueOntologyTermId.BRAIN.value)
suspension_type = str(SuspensionTypeId.SPATIAL.value)
tissue_type = "brain"




In [7]:
adata

AnnData object with n_obs × n_vars = 3227 × 315
    obs: 'slicename', 'x_CCF', 'y_CCF', 'z_CCF', 'mouse_name', '_scvi_batch', '_scvi_labels', 'leiden', 'n_counts'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'leiden', 'leiden_colors', 'log1p', 'mouse_name_colors', 'neighbors', 'slicename_colors', 'umap'
    obsm: 'X_scVI', 'X_umap', 'spatial'
    obsp: 'connectivities', 'distances'

In [8]:
#adata.X = csr_matrix(adata.X)
adata.obs[ObsConstants.SPATIAL_X] = adata.obs['z_CCF']
adata.obs[ObsConstants.SPATIAL_Y] = adata.obs['y_CCF']
adata.obs[ObsConstants.ASSAY_ONTOLOGY_TERM_ID] = pd.Categorical([assay for i in range(len(adata))])
adata.obs[ObsConstants.SEX_ONTOLOGY_TERM_ID] = pd.Categorical([sex for i in range(len(adata))])
adata.obs[ObsConstants.ORGANISM_ONTOLOGY_TERM_ID] = pd.Categorical([organism for i in range(len(adata))])
adata.obs[ObsConstants.TISSUE_ONTOLOGY_TERM_ID] = pd.Categorical([tissue for i in range(len(adata))])
adata.obs[ObsConstants.SUSPENSION_TYPE] = pd.Categorical([suspension_type for i in range(len(adata))])

adata.obs[ObsConstants.DONOR_ID] = pd.Categorical(['Xenium_Preview_Human_Non_diseased_Lung_With_Add_on_FFPE_outs' for i in range(len(adata))])
adata.obs[ObsConstants.CONDITION_ID] = pd.Categorical(['non diseased' for i in range(len(adata))])
adata.obs[ObsConstants.TISSUE_TYPE] = pd.Categorical([tissue_type for i in range(len(adata))])

adata.uns[UnsConstants.TITLE] = "Xenium_Preview_Human_Non_diseased_Lung_With_Add_on_FFPE_outs"
adata.var[VarConstants.FEATURE_IS_FILTERED] = False

adata.obs[ObsConstants.LIBRARY_KEY] = pd.Categorical(['section' for i in range(len(adata))])

In [26]:
adata_1 = ensem_for_adata_mer(adata)

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/ec2-user/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/ec2-user/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/ec2-user/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.pep.all.fa.gz.pickle


total invalid 1, from 315 0.31746031746031744 %
(3227, 315)
new data size (3227, 315)


In [16]:
from pyensembl import EnsemblRelease


In [18]:
data = EnsemblRelease(104, species='mus_musculus')    
data.index()

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/ec2-user/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/ec2-user/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/ec2-user/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.pep.all.fa.gz.pickle


In [25]:
[gene.gene_name for gene in data.genes()]

['Gnai3',
 'Pbsn',
 'Cdc45',
 'H19',
 'Scml2',
 'Apoh',
 'Narf',
 'Cav2',
 'Klf6',
 'Scmh1',
 'Cox5a',
 'Tbx2',
 'Tbx4',
 'Zfy2',
 'Ngfr',
 'Wnt3',
 'Wnt9a',
 'Fer',
 'Xpo6',
 'Tfe3',
 'Axin2',
 'Brat1',
 'Gna12',
 'Slc22a18',
 'Itgb2l',
 'Igsf5',
 'Pih1d2',
 'Dlat',
 'Sdhd',
 'Fgf23',
 'Fgf6',
 'Ccnd2',
 'Gpr107',
 'Nalcn',
 'Btbd17',
 'Slfn4',
 'Th',
 'Ins2',
 'Scnn1g',
 'Drp2',
 'Tspan32',
 'Lhx2',
 'Clec2g',
 'Gmpr',
 'Glra1',
 'Mid2',
 'Trim25',
 'Dgke',
 'Scpep1',
 'Mnt',
 'Itgb2',
 'Hddc2',
 'Tpd52l1',
 'Pemt',
 'Cdh1',
 'Cdh4',
 'Ckmt1',
 'Bcl6b',
 'Clec10a',
 'Alox12',
 'Comt',
 'Rtca',
 'Dbt',
 'Dazap2',
 'Mcts1',
 'Rem1',
 'Rnf17',
 'Trappc10',
 'Ccm2',
 'Wap',
 'Tbrg4',
 'Tmprss2',
 'Mx1',
 'Fap',
 'Gcg',
 'Ndufa9',
 'Egfl6',
 'Lck',
 'Tssk3',
 'Cttnbp2',
 'Galnt1',
 'Myf5',
 'Mkrn2',
 'Pparg',
 'Raf1',
 'Septin1',
 'Pdgfb',
 'Acvrl1',
 'Tamalin',
 'Acvr1b',
 'Tom1l2',
 'Gpa33',
 'Zfp385a',
 'Itga5',
 'Gabra2',
 'Wdr77',
 'Adora3',
 'Atp5pb',
 'Sox9',
 'Hnrnpd',
 'Dynlt1c',

In [27]:

adata_output, valid, errors, is_seurat_convertible = validate(adata_1, organism=organism_validator)

INFO:cellxgene_schema.validate:Starting validation...
INFO:cellxgene_schema.validate:Validation complete in 0:00:00.511865 with status is_valid=True
INFO:cellxgene_schema.validate:Writing labels
INFO:cellxgene_schema.utils:enforce canonical format in X
INFO:cellxgene_schema.utils:enforce canonical format in raw.X
INFO:cellxgene_schema.validate:H5AD label writing complete in 0:00:00.200056


In [28]:
preprocessed_path = f"/home/ec2-user/SageMaker/our_data_LC_NE/preprocessed"

In [29]:
adata_output.obs

,slicename,x_CCF,y_CCF,z_CCF,mouse_name,_scvi_batch,_scvi_labels,leiden,n_counts,x,...,tissue_ontology_term_id,suspension_type,donor_id,condition_id,tissue_type,library_key,assay,organism,sex,tissue
neuron_id,,,,,,,,,,,,,,,,,,,,,
110143770882700220040132120043038831763,MouseC_C2,407.519014,197.276290,266.738031,MouseC,0,0,4,972.0,266.738031,...,UBERON:0000955,na,Xenium_Preview_Human_Non_diseased_Lung_With_Ad...,non diseased,brain,section,MERFISH,Mus musculus,unknown,brain
113866178324333268659892041555653198479,MouseC_C2,407.194692,191.154244,268.527598,MouseC,0,0,7,638.0,268.527598,...,UBERON:0000955,na,Xenium_Preview_Human_Non_diseased_Lung_With_Ad...,non diseased,brain,section,MERFISH,Mus musculus,unknown,brain
120882274368051877867511817169740490916,MouseC_C2,405.119627,151.984390,256.980065,MouseC,0,0,1,131.0,256.980065,...,UBERON:0000955,na,Xenium_Preview_Human_Non_diseased_Lung_With_Ad...,non diseased,brain,section,MERFISH,Mus musculus,unknown,brain
123024757869921146658659796693384394846,MouseC_C2,404.933683,148.474425,249.260515,MouseC,0,0,4,185.0,249.260515,...,UBERON:0000955,na,Xenium_Preview_Human_Non_diseased_Lung_With_Ad...,non diseased,brain,section,MERFISH,Mus musculus,unknown,brain
12540531990191327531728358278060678344,MouseC_C2,404.842190,146.747376,252.804036,MouseC,0,0,1,2508.0,252.804036,...,UBERON:0000955,na,Xenium_Preview_Human_Non_diseased_Lung_With_Ad...,non diseased,brain,section,MERFISH,Mus musculus,unknown,brain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96119208577592618782594425212756632908,MouseZM_ZM7p2,424.618466,166.654299,194.961194,MouseZM,23,0,1,680.0,194.961194,...,UBERON:0000955,na,Xenium_Preview_Human_Non_diseased_Lung_With_Ad...,non diseased,brain,section,MERFISH,Mus musculus,unknown,brain
9623349862178950852359237543126573460,MouseZM_ZM7p2,424.358181,171.836997,192.938647,MouseZM,23,0,5,1160.0,192.938647,...,UBERON:0000955,na,Xenium_Preview_Human_Non_diseased_Lung_With_Ad...,non diseased,brain,section,MERFISH,Mus musculus,unknown,brain
96245750577479322584616409396904825217,MouseZM_ZM7p2,424.299410,171.174947,194.574443,MouseZM,23,0,1,1949.0,194.574443,...,UBERON:0000955,na,Xenium_Preview_Human_Non_diseased_Lung_With_Ad...,non diseased,brain,section,MERFISH,Mus musculus,unknown,brain


In [30]:
preprocessed_path

'/home/ec2-user/SageMaker/our_data_LC_NE/preprocessed'

In [31]:
adata_output.obs['assay'] = pd.Categorical(['MERFISH' for i in range(len(adata_output))])
adata_output.obs[ObsConstants.ASSAY_ONTOLOGY_TERM_ID] = pd.Categorical(['no yet defined' for i in range(len(adata_output))])

adata_output.obs[ObsConstants.DATASET] = adata_output.uns['title']
adata_output.obs[ObsConstants.SPLIT] = 'train'
adata_output.obs[ObsConstants.NICHE] = 'nan'
adata_output.obs[ObsConstants.REGION] = 'nan'

adata_output.write(f"{preprocessed_path}/adata_mer_pre.h5ad")

... storing 'dataset' as categorical
... storing 'nicheformer_split' as categorical
... storing 'niche' as categorical
... storing 'region' as categorical


In [32]:
adata.X.max()

66.0

In [24]:
adata_output